In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from google.auth import default
import itertools

In [ ]:
# Authenticate and create the gspread client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# STEP 1 - Load the Google Sheet (Published as CSV)
# ---------------------------------------------
# Replace with the name or URL of your Google Sheet
csv_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRF-FqJhbr-E4YGV2v43HZBFD009FR6wDBDgZIxzUWEv45omwyA_DG8dGdjXYoF_c355AA8h7s-4RVx/pub?gid=0&single=true&output=csv"

# Read the Sheet as real values
df = pd.read_csv(csv_url)

# Preview the raw data
print("Raw Data Preview:")
print(df.head(20))

Raw Data Preview:
    Partition  Insulation  Facade               Name Code Name  Thickness (m)  \
0         0.0         0.0     1.0     Concrete 30/37       CON        0.18000   
1         NaN         NaN     NaN     Concrete 20/25       CON        0.16000   
2         NaN         NaN     NaN     Concrete 25/30       CON        0.16000   
3         NaN         NaN     NaN     Concrete 25/30       CON        0.18000   
4         0.0         NaN     0.0              Earth       EAR        0.50000   
5         0.0         NaN     0.0              Earth       EAR        0.30000   
6         NaN         NaN     NaN              Straw       STR        0.40000   
7         1.0         NaN     0.0  Timber Mass (CLT)       TIM        0.24000   
8         NaN         NaN     NaN  Timber Mass (CLT)       TIM        0.22000   
9         NaN         NaN     NaN  Timber Mass (CLT)       TIM        0.14000   
10        1.0         0.0     0.0       Timber frame       TIF        0.12000   
11        

In [ ]:
# STEP 2 - Clean the Data
# ---------------------------------------------
# Keep only necessary columns
df_clean = df[['Name', 'Code Name', 'Thickness (m)', 'EW (Ext. Wall)', 'IW (Int. Wall)', 'ES (Ext.Slab)', 'IS (Int. Slab)', 'RO (Roof)', 'A-D/m²']]
df_clean.columns = ['Name', 'Code', 'Thickness', 'EW', 'IW', 'ES', 'IS', 'RO', 'A-D']

# Drop rows missing important values
df_clean = df_clean.dropna(subset=['Code', 'Thickness', 'A-D'])

# Convert to correct types
for col in ['Thickness', 'EW', 'IW', 'ES', 'IS', 'RO', 'A-D']:
    df_clean[col] = df_clean[col].astype(float)

print("Cleaned Data Preview:")
print(df_clean.head())
print(df_clean.size)

Cleaned Data Preview:
             Name Code  Thickness   EW   IW   ES   IS   RO       A-D
0  Concrete 30/37  CON       0.18  0.0  0.0  1.0  0.0  1.0  34.88292
1  Concrete 20/25  CON       0.16  0.0  1.0  0.0  0.0  0.0  24.67104
2  Concrete 25/30  CON       0.16  0.0  0.0  0.0  1.0  0.0  28.54304
3  Concrete 25/30  CON       0.18  1.0  0.0  0.0  0.0  0.0  32.11092
4           Earth  EAR       0.50  1.0  0.0  0.0  0.0  0.0  11.80370
135


In [ ]:
# STEP 3 - Set Surface Areas (input by user) ---- From BDG
# ---------------------------------------------
# Define surface areas for each building element (in m²)
EW_area = 1   # Example: External Wall area
IW_area = 1    # Example: Internal Wall area
ES_area = 1   # Example: External Slab area
IS_area = 1   # Example: Internal Slab area
RO_area = 1   # Example: Roof area

areas = {
    'EW': EW_area,
    'IW': IW_area,
    'ES': ES_area,
    'IS': IS_area,
    'RO': RO_area
}


In [ ]:
# STEP 4 - Prepare material categories
# ---------------------------------------------
materials = {
    'EW': df_clean[df_clean['EW'] == 1],
    'IW': df_clean[df_clean['IW'] == 1],
    'ES': df_clean[df_clean['ES'] == 1],
    'IS': df_clean[df_clean['IS'] == 1],
    'RO': df_clean[df_clean['RO'] == 1]
}

In [ ]:
# STEP 5 - Generate Combinations and Calculate Weighted Total A-D
# ---------------------------------------------
combinations = list(itertools.product(
    materials['EW'].index,
    materials['IW'].index,
    materials['ES'].index,
    materials['IS'].index,
    materials['RO'].index
))

results = []

for combo in combinations:
    elements = {
        'EW': df_clean.loc[combo[0]],
        'IW': df_clean.loc[combo[1]],
        'ES': df_clean.loc[combo[2]],
        'IS': df_clean.loc[combo[3]],
        'RO': df_clean.loc[combo[4]]
    }

    total_weighted_ad = 0
    code_parts = []
    name_parts = []

    # For each element (EW, IW, etc.)
    for element_type, material in elements.items():
        area = areas[element_type]
        ad_per_m2 = material['A-D']
        weighted_ad = ad_per_m2 * area

        total_weighted_ad += weighted_ad
        code_parts.append(material['Code'])
        name_parts.append(material['Name'])

    combo_code = '_'.join(code_parts)

    results.append({
        "Combination": ' | '.join(name_parts),
        "Code": combo_code,
        "Total Weighted A-D": total_weighted_ad
    })

In [ ]:
# STEP 6 - Create Final Result Table
# ---------------------------------------------
result_df = pd.DataFrame(results)

# Optional: Remove duplicate codes
result_df = result_df.drop_duplicates(subset=['Code'])

print("Generated Typology Combinations (with Weighted A-D):")
print(result_df)

Generated Typology Combinations (with Weighted A-D):
                                           Combination                 Code  \
0    Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_CON   
1    Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_TIM   
2    Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_CON_TIF   
3    Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_TIM_CON   
4    Concrete 25/30 | Concrete 20/25 | Concrete 30/...  CON_CON_CON_TIM_TIM   
..                                                 ...                  ...   
319  Brick | Brick | Concrete 30/37 | Timber Mass (...  BRI_BRI_CON_TIM_TIM   
320  Brick | Brick | Concrete 30/37 | Timber Mass (...  BRI_BRI_CON_TIM_TIF   
321  Brick | Brick | Concrete 30/37 | Timber frame ...  BRI_BRI_CON_TIF_CON   
322  Brick | Brick | Concrete 30/37 | Timber frame ...  BRI_BRI_CON_TIF_TIM   
323  Brick | Brick | Concrete 30/37 | Timber frame ...  BRI_BRI_CON_TIF_TIF   

In [ ]:
# STEP 7 - Save to CSV
# ---------------------------------------------
result_df.to_csv('typology_combinations_weighted.csv', index=False)

print("✅ Typology combinations with weighted A-D saved as 'typology_combinations_weighted.csv'")

# (Optional) Easy Download
from google.colab import files
files.download('typology_combinations_weighted.csv')

✅ Typology combinations with weighted A-D saved as 'typology_combinations_weighted.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>